<a href="https://colab.research.google.com/github/praffuln/agentic-ai/blob/master/crew_ai_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('hello world!!')

In [ ]:
!python3 --version


Python 3.11.11


In [ ]:
!pip install 'crewai[tools]' google-generativeai


## Setup Keys For GOOGLE_API_KEY


In [ ]:

 # Assigning value to variable
GEMINI_API_KEY=''
SERPER_API_KEY = ''



# setup environments


In [ ]:
import os
os.environ['SERPER_API_KEY'] = SERPER_API_KEY
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY




## setup LLM

In [ ]:
from crewai import Agent, Task, Process, Crew, LLM

In [ ]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("Explain how AI works")
print(response.text)


In [ ]:
llm = LLM(
    model="gemini/gemini-1.5-flash",
    temperature=0.7
)

In [ ]:
from crewai import Agent
from crewai_tools import SerperDevTool


research_agent = Agent(
    role="Research Analyst",
    goal="Find and summarize information about specific topics",
    backstory="You are an experienced researcher with attention to detail",
    tools=[SerperDevTool()],
    llm=llm,
    verbose=True,  # Enable logging for debugging
)

project_description = "a clock on the webpage"

requirement_creator_agent = Agent(
    role="Requirement Creator",
    goal="Find the requirement for given project based on {project_description}",
    backstory="You are an excillent in search and able to find functional requirement based on project_description",
    tools=[SerperDevTool()],
    verbose=True,  # Enable logging for debugging
    llm=llm
)




In [ ]:
requirement_creator_task = Task(
    name="Requirement Creator",
    description="This Task creates Functional Requirements based on given {project_description}",
    expected_output="""at least a description of functional requirement which is asked by agent.
                    List your output as points, you can use below example
                    '''
                          1. **Search Functionality:**
                                * The system shall accept user input (textual queries) via a search bar.
                                * The system shall process user queries and generate relevant search results using a generative AI model.  This includes understanding the intent and context of the query, even if it's phrased in an unconventional way.
                                * The system shall handle various query types, including keyword searches, question-answering, and complex requests.
                                * The system shall support auto-completion and suggestion features to assist users in formulating their queries.
                            2. **Result Presentation:**
                                * The system shall display search results in a clear, concise, and user-friendly manner.
                                * The system shall present diverse result formats, such as summaries, links to relevant web pages, images, videos, and other multimedia content.
                            3. **User Interface:**
                                * The system shall have a visually appealing and intuitive user interface.
                                * The system shall be accessible to users with disabilities, conforming to WCAG guidelines.
                            4. **Integration:**
                                * The system may integrate with other services or APIs (e.g., knowledge graphs, external databases) to enhance search results and functionality.
                                * The system may allow users to save or share search results.
                            5. **Performance and Scalability:**
                                * The system shall be designed for high availability and scalability to handle a large volume of search queries.
                                * The system shall provide quick response times for search queries.
                    '''
                    """, # Added expected output
    agent=requirement_creator_agent  # Assign the requirement_creator_agent to the task
)




# UserStories Creator Agent and Tasks

In [ ]:
user_stories_creator_agent = Agent(
    role="User Stories Creator",
    goal="Create User Stories for sprint-1 based on requirements",
    backstory="You are an excillent in creation of user-stories and able to find create concise user stories with descriptions",
    tools=[SerperDevTool()],
    verbose=True,  # Enable logging for debugging
    llm=llm
)


user_stories_creator_task = Task(
    name="User Stories Creator",
    description="This Task creates userstories based on given requirements",
    expected_output="""create the userstories for first sprint of given requirements
                       - User stories must have proper description should be at least 2 lines and not more than 6 lines.
                       - User stories must have proper acceptance criteria
                       - User stories should be completely functional
                       - User stories should be understandable to anyone including scrum-master, project-owner and developers
                    """, # Added expected output
    agent=user_stories_creator_agent  # Assign the requirement_creator_agent to the task
)





# Story Points Estimator Agent and Tasks

In [ ]:
story_point_estimator_agent = Agent(
    role="Story Point Estimator",
    goal="Estimate the Story Points based on complexity",
    backstory="""You are an excillent in estimate the user stories in any scrum, estimate the stories points based on
                  - Story complexity
                  - Story efforts
                  - Story Functionality
                  - Story Development Efforts
                  - Story Testing Efforts
                  - Story Priority
                  - Story descriptions
              """,
    verbose=True,  # Enable logging for debugging
    llm=llm
)


story_point_estimator_task = Task(
    name="User Stories Creator",
    description="This Task estimates story points",
    expected_output="""
                      estimation of story points using a Fibonacci sequence (1, 2, 3, 5, 8, 13...) to represent complexity.
                      Estimate the stories points based on
                        - Story complexity
                        - Story efforts
                        - Story Functionality
                        - Story Development Efforts
                        - Story Testing Efforts
                        - Story Priority
                        - Story descriptions
                      Print your output in table format
                      Also print reason for that story point
                    """, # Added expected output
    agent=story_point_estimator_agent  # Assign the story_point_estimator_agent to the task
)





# Story tasks creation Agent and task

In [ ]:
story_task_creation_agent = Agent(
    role="Task Creator",
    goal="create the story task under each story",
    backstory="""You are an excillent in task creation of any given user story, create the task based on userstories description.
              """,
    verbose=True,  # Enable logging for debugging
    llm=llm
)


story_task_creation_task = Task(
    name="Task Creator",
    description="This Task creates task under user stories",
    expected_output="""
                      - create the task based on user-stories description, acceptance criteria and story points.
                      - tasks should be # seprated
                      - Assign task between kesava, nirmal, asif, tanmaya
                      - Print your output in table format.
                    """, # Added expected output
    agent=story_task_creation_agent  # Assign the story_point_estimator_agent to the task
)





# Crew tasks and agents

In [ ]:
# Create a Crew and add the agent and task directly in the initialization
crew = Crew(agents=[requirement_creator_agent, user_stories_creator_agent, story_point_estimator_agent, story_task_creation_agent],
            tasks=[requirement_creator_task, user_stories_creator_task, story_point_estimator_task, story_task_creation_task],
            process=Process.sequential)

# Execute the task using the crew
print(project_description)
result = crew.kickoff(inputs={"project_description" : project_description})

# Print the result
print(result)


a clock on the webpage
# Agent: Requirement Creator
## Task: This Task creates Functional Requirements based on given a clock on the webpage


# Agent: Requirement Creator
## Thought: Thought: I need to find functional requirements related to a clock displayed on a webpage.  I will search the internet for information on web clock functionalities and user requirements.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"functional requirements for a webpage clock\"}"
## Tool Output: 
{'searchParameters': {'q': 'functional requirements for a webpage clock', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Solved Functional Requirements 1. Clock12: Time should be - Chegg', 'link': 'https://www.chegg.com/homework-help/questions-and-answers/functional-requirements-1-clock12-time-displayed-12-hour-format-clock-exceed-12-59-59-2-cl-q51074750', 'snippet': '1. Clock12: Time should be displayed in 12-hour format where the clock does not exceed 12:59:5